In [1]:
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import pandas as pd

import pickle as pkl
import re
import json
from collections import Counter

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

import autokeras as ak # !pip install autokeras

from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)

# import datetime
# import hvplot

2023-02-05 15:41:08.599133: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-05 15:41:09.097960: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ivv101/arrayfire/lib64:/usr/local/cuda/lib64:/home/ivv101/arrayfire/lib64:/usr/local/cuda/lib64::/home/ivv101/miniconda3/envs/p310/lib/:/home/ivv101/miniconda3/envs/p310/lib/
2023-02-05 15:41:09.098057: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
# %env CLEARML_WEB_HOST=https://app.clear.ml
# %env CLEARML_API_HOST=https://api.clear.ml
# %env CLEARML_FILES_HOST=https://files.clear.ml
# %env CLEARML_API_ACCESS_KEY=Z0Z7YQU3HMJI84E6VJFE
# %env CLEARML_API_SECRET_KEY=GLBnwfxSOb2lzWDibL9uHR5FQwg5bAKFxBllsYMUwB8ljyZtrl

In [3]:
# from clearml import Task

In [4]:
# task = Task.init(project_name='great project', task_name='best experiment')

In [5]:
# %load_ext tensorboard

In [6]:
# rm -rf ./logs/

In [7]:
# log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [8]:
# %tensorboard --logdir logs/fit --host localhost

In [9]:
SEED = 321 #165 #321 #434

In [10]:
# if not os.path.exists(out_dir):
#     os.mkdir(out_dir)

In [11]:
log_features = ['Fs', 'Fm', 'Fh', 'Fb', 
                'rgeo', 'b_rgeo', 'B_rgeo',
                'PM'
               ]

features = ['Fs', 'Fm', 'Fh', 'Fb',
            'P_intra', 'P_inter',
            'Gmag', 'BPmag', 'RPmag', 'Jmag', 'Hmag', 'Kmag',
            'W1mag', 'W2mag', 'W3mag',
            'Plx', 'PM',
            'rgeo', 'b_rgeo', 'B_rgeo'
           ]

errors = ['e_Fs', 'e_Fm', 'e_Fh', 'e_Fb',
          'e_Gmag', 'e_BPmag', 'e_RPmag', 'e_Jmag', 'e_Hmag', 'e_Kmag',
          'e_W1mag', 'e_W2mag', 'e_W3mag', 
          'e_Plx']

In [12]:
df = pd.read_csv('TD.csv')
df[log_features] = df[log_features].apply(np.log10)
df

,CSCv2_name,CSCv2_RA,CSCv2_DEC,CSCv2_PU,CSCv2_S/N,Fs,e_Fs,Fm,e_Fm,Fh,...,e_P_NS,e_P_YSO,Class,Class_prob,e_Class_prob,CT,CSCv2_flags,Catalog_name,True_Class,Class_ref
0,2CXO J000009.3+135618,0.039115,13.938493,0.79,1.95,-14.698970,1.000000e-15,-15.301030,5.000000e-16,-14.698970,...,0.0040,0.0500,AGN,0.900,0.100,5.307,NaN,SDSS J00001+1356,AGN,2010A&A...518A..10V
1,2CXO J000230.7+004959,0.627974,0.833072,0.72,11.07,-13.244125,7.000000e-15,-13.537602,5.000000e-15,-13.221849,...,0.0003,0.0004,AGN,0.999,0.004,145.799,NaN,PB 5698,AGN,2010A&A...518A..10V
2,2CXO J000622.6-000424,1.594324,-0.073573,0.78,25.42,-12.823909,1.000000e-14,-12.931814,8.000000e-15,-12.397940,...,0.0050,0.0009,AGN,0.970,0.020,20.820,extent,3C 2.0,AGN,2010A&A...518A..10V
3,2CXO J000659.2-001740,1.747051,-0.294661,0.84,4.11,-13.958607,4.000000e-15,-14.221849,3.000000e-15,-13.853872,...,0.0030,0.0070,AGN,0.950,0.060,8.318,NaN,SDSS J00069-0017,AGN,2010A&A...518A..10V
4,2CXO J000703.6+155423,1.765007,15.906575,0.72,8.63,-14.301030,3.000000e-15,-13.795880,6.000000e-15,-12.267606,...,0.0004,0.0090,AGN,0.990,0.010,47.057,NaN,2MASS J00070+1554,AGN,2010A&A...518A..10V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2936,2CXO J183000.6+011340,277.502734,1.227794,0.71,20.82,-14.657577,4.000000e-16,-13.795880,1.000000e-15,-13.200659,...,0.0010,0.0300,YSO,0.930,0.030,16.735,NaN,2MASS J18300065+0113402,YSO,2007A&A...463..275G
2937,2CXO J183001.1+011324,277.504621,1.223367,0.75,8.10,-16.154902,7.000000e-17,-15.000000,3.000000e-16,-13.795880,...,0.0010,0.0900,YSO,0.800,0.090,3.192,NaN,2MASS J18300111+0113242,YSO,2007A&A...463..275G
2938,2CXO J183003.4+011619,277.514213,1.271981,0.73,13.69,-14.508638,5.000000e-16,-14.136677,8.000000e-16,-13.744727,...,0.0000,0.0050,YSO,0.998,0.005,124.848,NaN,2MASS J18300341+0116191,YSO,2007A&A...463..275G
2939,2CXO J183007.7+011204,277.532159,1.201211,0.75,15.60,-14.420216,6.000000e-16,-14.000000,1.000000e-15,-13.619789,...,0.0000,0.0070,YSO,0.996,0.007,84.887,extent,2MASS J18300770+0112043,YSO,2007A&A...463..275G


In [13]:
# {f: [df[f].min(), df[f].max()] for f in features}

In [14]:
df_feat = df[features].apply(lambda x : (x-x.mean())/x.std())

In [15]:
{f: [df_feat[f].min(), df_feat[f].max()] for f in features}

{'Fs': [-2.576097051867796, 3.037547888651721],
 'Fm': [-3.1633175259424413, 3.353068673263792],
 'Fh': [-3.167529234962109, 3.6737353736479137],
 'Fb': [-2.359584277219483, 4.031512746439284],
 'P_intra': [-2.292931596557009, 0.9472948406444076],
 'P_inter': [-3.0385115023161156, 0.6455515963566589],
 'Gmag': [-4.196046959939427, 1.5323452993212447],
 'BPmag': [-4.0727319624873894, 1.731833276730117],
 'RPmag': [-4.124723888688924, 1.6909016342756864],
 'Jmag': [-4.425041079460414, 2.124007095008807],
 'Hmag': [-4.71265263641138, 2.3305540821294275],
 'Kmag': [-4.862626877029222, 2.3155364970244823],
 'W1mag': [-4.481634646882916, 1.9060303148077884],
 'W2mag': [-4.555152917342131, 2.0219436807633633],
 'W3mag': [-6.329868656158829, 1.6005678318859842],
 'Plx': [-2.4774673348866676, 13.867643841820009],
 'PM': [-3.5940296284202304, 2.597799702665246],
 'rgeo': [-3.0630369317925497, 2.322108818082452],
 'b_rgeo': [-3.20847676381954, 2.6005180022943195],
 'B_rgeo': [-2.9034591466815143,

In [16]:
# bins = np.arange(df_feat.min().min(), df_feat.max().max(), 0.1)

# df_dig = df_feat.apply(lambda x : np.digitize(x, bins))
# df_dig[df_feat.isna()] = 0

# tf_dig = tf.constant(df_dig, dtype=tf.int32)

# len(bins)

In [17]:
print((df_feat.isna().sum()/len(df_feat)*100))

Fs          0.000000
Fm          0.000000
Fh          0.000000
Fb          0.000000
P_intra     7.446447
P_inter    54.403264
Gmag       24.311459
BPmag      25.127508
RPmag      24.957497
Jmag       37.980279
Hmag       37.980279
Kmag       37.980279
W1mag      23.937436
W2mag      24.651479
W3mag      38.796328
Plx        32.709963
PM         32.709963
rgeo       32.709963
b_rgeo     32.709963
B_rgeo     32.709963
dtype: float64


In [18]:
Counter(df['True_Class'])

Counter({'AGN': 1390,
         'CV': 44,
         'HM-STAR': 118,
         'YSO': 1004,
         'HMXB': 26,
         'LMXB': 65,
         'LM-STAR': 207,
         'NS': 87})

In [19]:
mlb = LabelBinarizer()

y_true = mlb.fit_transform(df['True_Class'])
y_pred_hui = mlb.transform(df['Class'])

classes = mlb.classes_

# classes, labels.shape

print(classification_report(y_true, y_pred_hui, 
                            output_dict=False,
                            target_names=classes))

              precision    recall  f1-score   support

         AGN       0.98      0.93      0.95      1390
          CV       0.48      0.55      0.51        44
     HM-STAR       0.67      0.72      0.69       118
        HMXB       0.75      0.69      0.72        26
     LM-STAR       0.81      0.86      0.84       207
        LMXB       0.19      0.26      0.22        65
          NS       0.51      0.68      0.58        87
         YSO       0.93      0.94      0.93      1004

   micro avg       0.89      0.89      0.89      2941
   macro avg       0.67      0.70      0.68      2941
weighted avg       0.90      0.89      0.89      2941
 samples avg       0.89      0.89      0.89      2941



In [20]:
# tf_y_true = tf.constant(y_true, dtype=tf.int32)
# tf_y_true

In [21]:
# model.get_config()

In [22]:
# mod.get_config()

In [26]:
df_feat

,Fs,Fm,Fh,Fb,P_intra,P_inter,Gmag,BPmag,RPmag,Jmag,Hmag,Kmag,W1mag,W2mag,W3mag,Plx,PM,rgeo,b_rgeo,B_rgeo
0,-0.199449,-1.033569,-0.741320,-0.973895,0.697797,NaN,0.492168,0.535112,0.746560,NaN,NaN,NaN,0.853763,0.826402,0.487181,-0.742511,-1.095253,1.198046,1.122459,1.536628
1,1.242478,1.057694,0.753216,0.928074,-1.262540,NaN,0.288212,0.335138,0.531072,1.445468,1.631887,1.702818,0.548266,0.363090,0.268411,-0.514228,-1.089332,0.626017,0.611672,0.782127
2,1.658963,1.776103,1.586837,1.765626,-2.046674,NaN,0.969920,0.969431,1.193379,NaN,NaN,NaN,0.795132,0.706524,0.643446,-0.651198,-2.149547,0.370039,0.200022,0.403904
3,0.534340,0.246240,0.113741,0.121539,-0.290472,NaN,0.722391,0.706965,1.015919,NaN,NaN,NaN,1.180861,1.163357,NaN,-1.107765,-0.699429,0.800462,0.759952,0.929681
4,0.194958,0.751400,1.718708,1.669686,-1.855501,NaN,0.493095,0.407004,0.337767,0.820297,0.852450,0.694386,-0.343539,-0.476057,-0.731684,-0.559884,-0.458692,0.897704,0.625411,0.868525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2936,-0.158424,0.751400,0.774655,0.635347,0.947295,NaN,0.985371,1.263144,0.819445,0.096013,-0.407573,-0.599130,-0.942189,-0.916690,-0.419154,0.535877,1.124639,-0.078631,-0.521381,0.104488
2937,-1.642453,-0.676575,0.172417,-0.199851,0.947295,NaN,NaN,NaN,NaN,1.838106,1.304766,0.719916,-0.436114,-0.702853,-0.790283,NaN,NaN,NaN,NaN,NaN
2938,-0.010807,0.347246,0.224173,0.067089,0.947295,NaN,-0.176562,0.275771,-0.232641,-0.315685,-0.451997,-0.488500,-0.929846,-0.929650,-1.169225,0.357816,1.145793,-0.862243,-0.774274,-0.921537
2939,0.076830,0.509333,0.350584,0.230456,0.292769,NaN,0.145133,0.816326,0.005029,-0.327121,-0.536806,-0.620405,-1.081052,-1.149966,-1.149692,0.307594,1.262462,-0.782186,-0.755222,-0.788375


In [23]:
import autosklearn.classification
cls = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder="/tmp/autosklearn_classification_example_tmp",
)

In [27]:
autosklearn.classification.AutoSklearnClassifier?

Init signature:
autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=3600,
    per_run_time_limit=None,
    initial_configurations_via_metalearning=25,
    ensemble_size: 'int | None' = None,
    ensemble_class: "Type[AbstractEnsemble] | Literal['default'] | None" = 'default',
    ensemble_kwargs: 'Dict[str, Any] | None' = None,
    ensemble_nbest=50,
    max_models_on_disc=50,
    seed=1,
    memory_limit=3072,
    include: 'Optional[Dict[str, List[str]]]' = None,
    exclude: 'Optional[Dict[str, List[str]]]' = None,
    resampling_strategy='holdout',
    resampling_strategy_arguments=None,
    tmp_folder=None,
    delete_tmp_folder_after_terminate=True,
    n_jobs: 'Optional[int]' = None,
    dask_client: 'Optional[dask.distributed.Client]' = None,
    disable_evaluator_output=False,
    get_smac_object_callback=None,
    smac_scenario_args=None,
    logging_config=None,
    metadata_directory=None,
    metric: 'Scorer | Sequence[Scorer] | None' = None,
    s

In [24]:
cls.fit(df_feat, y_true)

[ERROR] [2023-02-05 15:43:27,726:Client-AutoML(1):8c4a17e8-a595-11ed-8388-9061aedbab23] 'generator' object is not subscriptable
Traceback (most recent call last):
  File "/home/ivv101/miniconda3/envs/p310/lib/python3.10/site-packages/autosklearn/automl.py", line 899, in fit
    ) = _proc_smac.run_smbo()
  File "/home/ivv101/miniconda3/envs/p310/lib/python3.10/site-packages/autosklearn/smbo.py", line 446, in run_smbo
    metalearning_configurations = self.get_metalearning_suggestions()
  File "/home/ivv101/miniconda3/envs/p310/lib/python3.10/site-packages/autosklearn/smbo.py", line 638, in get_metalearning_suggestions
    meta_base = MetaBase(
  File "/home/ivv101/miniconda3/envs/p310/lib/python3.10/site-packages/autosklearn/metalearning/metalearning/meta_base.py", line 49, in __init__
    aslib_reader = aslib_simple.AlgorithmSelectionProblem(
  File "/home/ivv101/miniconda3/envs/p310/lib/python3.10/site-packages/autosklearn/metalearning/input/aslib_simple.py", line 35, in __init__
    

TypeError: 'generator' object is not subscriptable

In [ ]:
predictions = cls.predict(X_test)

In [ ]:
# df_train, df_test = train_test_split(df, test_size=0.2, random_state=SEED)

In [ ]:
METRICS = [
    keras.metrics.CategoricalAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
]

In [ ]:
df_inp = df_feat.copy()
df_inp[df_inp.isna()] = 0

In [ ]:
# use masking nans for RNN such as LSTM!!!
# https://www.tensorflow.org/guide/keras/masking_and_padding

dense_units = [16]*3

dropout_units = [0.3]*3

inputs = keras.layers.Input(shape=(20,))

x = inputs

# x = base_model.layers[1](x)
# x = base_model.layers[2](x)

x = keras.layers.Normalization()(x)

x = keras.layers.Dense(dense_units[0], activation='relu')(x)

x = keras.layers.Dropout(dropout_units[0])(x)

x = keras.layers.Dense(dense_units[1], activation='relu')(x)

x = keras.layers.Dropout(dropout_units[1])(x)

x = keras.layers.Dense(dense_units[2], activation='relu')(x)

x = keras.layers.Dropout(dropout_units[2])(x)

x = keras.layers.Dense(8, activation='softmax')(x)

model = keras.Model(inputs, x)

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=METRICS)

In [ ]:
# model.fit?

In [ ]:
# model.fit?

In [ ]:
%%time

hist = model.fit(df_inp.to_numpy(), y_true,
                 epochs=100, 
                 batch_size=32, 
                 # validation_data=(padded_docs_test, y_test),
                     # verbose=1,
                validation_split=0.2
                    )

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Baseline Embedding model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper right')
plt.show()

In [ ]:
y_pred_mod = model.predict(df_inp.to_numpy())

In [ ]:
print(classification_report(y_true, y_pred_mod.round(), 
                            output_dict=False,
                            target_names=classes))

In [ ]:
y_pred_mod.round()

In [ ]:
%%time

clf = ak.StructuredDataClassifier(
    overwrite=True, 
    max_trials=1,
    metrics=METRICS,
    seed=SEED
)

history = clf.fit(
    df_inp,
    y_true,
    epochs=100,
    # callbacks=[tensorboard_callback]
    # verbose=0
)

In [ ]:
history.model.summary()

In [ ]:
# ak.ClassificationHead?

In [ ]:
# ak.ClassificationHead?

In [ ]:
df_dig.to_numpy()[0]

In [ ]:
rnn_mod.export_model().predict(df_dig.to_numpy()[:10])

In [ ]:
# df_feat.isna()

In [ ]:
ak_model = ak.TextClassifier(
    multi_label=False,
    overwrite=True, 
    max_trials=1,
    project_name='ak_model_14_vanilla',
    metrics=METRICS,
    seed=SEED
) 

In [ ]:
# ak.TextClassifier?

In [ ]:
# z = ak.Embedding(max_features=20001, pretraining=None, embedding_dim=None, dropout=None, mask_zero=True)

In [ ]:
ak_model.export_model().summary()

In [ ]:
data_str = np.array([' '.join(_) for _ in df_dig.to_numpy().astype(str)])

In [ ]:
%time history = ak_model.fit(data_str, y_true, epochs=100)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Baseline Embedding model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper right')
plt.show()

In [ ]:
predicted_y = mlb.transform(predicted_y)

In [ ]:
y_test_true = [[_] for _ in df_test['True_Class']]
y_test_true = mlb.transform(y_test_true)

In [ ]:
print(classification_report(y_test_true, predicted_y, 
                            target_names=classes))

In [ ]:
model = keras.Sequential(
    [layers.Embedding(len(bins)+1, 64, input_length=df_dig.shape[1], mask_zero=True), 
     layers.LSTM(128),
     layers.Dropout(0.2),
     layers.Dense(len(classes), activation='softmax')
    ]
)

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=METRICS)

In [ ]:
# model.fit?

In [ ]:
# layers.Embedding?

In [ ]:
%%time

hist = model.fit(df_dig, y_true,
                 epochs=10, 
                 batch_size=32, 
                 # validation_data=(padded_docs_test, y_test),
                     # verbose=1,
                validation_split=0.2
                    )

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Baseline Embedding model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper right')
plt.show()

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(len(bins)+1, 1, input_length=df_dig.shape[1], mask_zero=True))

In [ ]:
# input_array = np.random.randint(1000, size=(32, 10))
model.compile('rmsprop', 'mse')
output_array = model.predict(df_dig)
print(output_array.shape)

In [ ]:
%%time

clf = ak.StructuredDataClassifier(
    overwrite=True, 
    max_trials=1,
    metrics=METRICS,
    seed=SEED
)

history = clf.fit(
    df_feat,
    y_true,
    epochs=100,
    # callbacks=[tensorboard_callback]
    # verbose=0
)

In [ ]:
base_model = clf.export_model()

In [ ]:
# %tensorboard --logdir logs/fit --host localhost

In [ ]:
base_model.summary(expand_nested=True, show_trainable=True)

In [ ]:
base_model.get_config().keys()

In [ ]:
[_['config'] for _ in base_model.get_config()['layers']]

In [ ]:
# ak.StructuredDataInput?

In [ ]:
# keras.layers.Dense?

In [ ]:
inputs = keras.layers.Input(shape=(20,), dtype='float64')
x = inputs
x = base_model.layers[1](x)
x = base_model.layers[2](x)

mod_input = keras.Model(inputs, x)

In [ ]:
_ = df_feat.to_numpy() 
_, _.min(), _.max()

In [ ]:
inp = mod_input.predict(df_feat)
inp, inp.shape, inp.min(), inp.max()

In [ ]:
d = df_feat.to_numpy().astype(np.float32)
d, d.shape

In [ ]:
np.where(np.isnan(d))

In [ ]:
d[0, 5], inp[0, 5]

In [ ]:
inp[0]

In [ ]:
# use masking nans for RNN such as LSTM!!!
# https://www.tensorflow.org/guide/keras/masking_and_padding

inputs = keras.layers.Input(shape=(20,))

x = inputs

# x = base_model.layers[1](x)
# x = base_model.layers[2](x)

x = keras.layers.Normalization()(x)

x = keras.layers.Dense(32, activation='relu')(x)

x = keras.layers.Dropout(0.2)(x)

x = keras.layers.Dense(32, activation='relu')(x)

x = keras.layers.Dropout(0.2)(x)

# x = keras.layers.Dense(32, activation='relu')(x)

# x = keras.layers.Dropout(0.2)(x)

x = keras.layers.Dense(8, activation='softmax')(x)

model = keras.Model(inputs, x)

In [ ]:
keras.layers.Dropout?

In [ ]:
# tf.keras.losses.CategoricalCrossentropy?

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=METRICS)

In [ ]:
# model.fit?

In [ ]:
%%time

hist = model.fit(df_feat.to_numpy().astype(np.float32), y_true,
                 epochs=100, 
                 batch_size=32, 
                 # validation_data=(padded_docs_test, y_test),
                     # verbose=1,
                validation_split=0.2
                    )

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Baseline Embedding model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper right')
plt.show()

In [ ]:
input_layer = ak.StructuredDataInput()

x = input_layer

x = ak.Normalization()(x)

x = ak.DenseBlock(num_layers=1, num_units=32, activation='relu')(x)

x = ak.DenseBlock(num_layers=1, num_units=32, activation='relu')(x)

x = ak.DenseBlock(num_layers=1, num_units=8, activation='softmax')(x)

clf2 = ak.AutoModel(
    inputs=input_layer,
    outputs=x
)

In [ ]:
%%time

history = clf.fit(
    df_feat,
    y_pred,
    epochs=100,
    callbacks=[tensorboard_callback]
    # verbose=0
)

In [ ]:
mod = clf.export_model()

In [ ]:
mod.summary(expand_nested=True, show_trainable=True)

In [ ]:
mod.get_config()

In [ ]:
# import tensorflow as tf
from tensorflow.keras import layers

def build_model(input_shape=(20,), dtype='float32'):
    
    input_layer = layers.Input(shape=input_shape, name='input_1')
    
    # if encoding is not None:
    #     multi_category_encoding = CustomMultiCategoryEncoding(encoding=encoding, name='multi_category_encoding')
    #     x = multi_category_encoding(input_layer)
    # else:
    x = input_layer
        
    x = layers.Normalization(axis=(-1,), name='normalization')(x)
    x = layers.Dense(32, activation='relu', name='dense')(x)
    # x = layers.ReLU(dtype=dtype, name='re_lu')(x)
    x = layers.Dense(32, activation='relu', name='dense_1')(x)
    # x = layers.ReLU(dtype=dtype, name='re_lu_1')(x)
    x = layers.Dense(8, activation='softmax', name='dense_2')(x)
    
    model = tf.keras.Model(inputs=input_layer, outputs=x, name='model')
    
    return model

model = build_model()

In [ ]:
model.evaluate(df_feat.iloc[0])

In [ ]:
# tf.keras.optimizers.SGD(
#     learning_rate=0.0001, momentum=0.7, nesterov=False, name='sgd'
# )

model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=METRICS)

In [ ]:
model.optimizer.learning_rate

In [ ]:
df_feat.shape, y_true.shape

In [ ]:
%%time

hist = model.fit(df_feat, y_true,
                 epochs=10, 
                 batch_size=8, 
                 # validation_data=(padded_docs_test, y_test),
                     # verbose=1,
                     validation_split=0.2
                    )

In [ ]:
y_true = [[0, 1, 0], [0, 0, 1]]
y_pred = [[0., 0., 1.], [1., 1., 0.]] #[[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
# Using 'auto'/'sum_over_batch_size' reduction type.
cce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
cce(y_true, y_pred).numpy()

In [ ]:
tf.keras.losses.CategoricalCrossentropy??

In [ ]:
# Predict with the best model.
predicted_y = clf.predict(df_test[features])

In [ ]:
predicted_y = mlb.transform(predicted_y)

In [ ]:
y_test_true = [[_] for _ in df_test['True_Class']]
y_test_true = mlb.transform(y_test_true)

In [ ]:
print(classification_report(y_test_true, predicted_y, 
                            target_names=classes))

In [ ]:
# classification_report?

In [ ]:
Counter(df_test['True_Class'])

In [ ]:
df_train

In [ ]:
input_df = df[features].copy()
df2['label'] = df['True_Class']
# df2['old_pred'] = df['Class']

df2.index = df['CSCv2_name']

df2

In [ ]:
mlb = MultiLabelBinarizer()

labels = mlb.fit_transform([[_] for _ in df['True_Class']])
classes = mlb.classes_

classes, labels.shape

In [ ]:
labels

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df[features].to_numpy(), labels, test_size=0.2, random_state=SEED)

In [ ]:
train_test_split?

In [ ]:
Counter(df2['label'])

In [ ]:
old_pred = mlb.transform([[_] for _ in df2['old_pred']])
old_pred

In [ ]:
# classification_report?

In [ ]:
print(classification_report(labels, 
                            old_pred, 
                            output_dict=False,
                            target_names=classes))

In [ ]:
df2['label'].to_list()

In [ ]:
df = pkl.load(open(f'{tmp_dir}/data.pkl', 'rb'))

In [ ]:
def remove_labels(df, labels, in_col='genres', out_col='genres2'):
    
    if type(labels)==str:
        labels = [labels]
        
    df[out_col] = [sorted(list(set(_).difference(set(labels)))) for _ in df[in_col]]
    
    return df

### Let us remove genres that have <1000 entries as well as 'Musical', 'Mystery', and 'Biography' (they were found to be poorly perfoming in training, which makes sense).

In [ ]:
labels, abund = np.unique(np.concatenate(df['genres'].tolist()), return_counts=True)
        
to_remove = set(labels[np.where(abund<1000)[0]])
to_remove = to_remove.union({'Musical', 'Mystery', 'Biography'})

to_remove

In [ ]:
df = remove_labels(df, to_remove)
df

In [ ]:
# from collections import Counter
# Counter([len(_) for _ in df['genres2']])

### Binarizing labels for multi-label classification with Autokeras' TextClassifier model. It appears to be the simplest method that provided good f1 micro scores. Also, it does not require a heavy cleaning of the input such as stemming.

### Other models could include Bidirectional LSTM layers, or RoBERTa transformer model. Also one could fine-tune a pretrained model that calculates a semantic similarity between texts on pairs (synopsis, genres).

In [ ]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(df['genres2'])
classes = mlb.classes_

classes, labels.shape

In [ ]:
text_input = df['clean'].to_numpy()

x_train, x_test, y_train, y_test = train_test_split(text_input, labels, test_size=0.2, random_state=SEED)

In [ ]:
METRICS = [
    keras.metrics.CategoricalAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
]

In [ ]:
ak_model = ak.TextClassifier(
    multi_label=True,
    overwrite=True, 
    max_trials=1,
    project_name='ak_model_14_vanilla',
    metrics=METRICS,
    seed=SEED
) 

In [ ]:
%time history = ak_model.fit(x_train, y_train, epochs=10)

In [ ]:
tf_model = ak_model.export_model()
tf_model.summary()

In [ ]:
y_pred = tf_model.predict(x_test)

### We chose a threshold of 0.95 for a prediction array normalized by its maximum value in order to allow for multiple genres

In [ ]:
new_y_pred = np.where((y_pred.T/y_pred.max(1)).T > 0.95, 1, 0)

In [ ]:
from collections import Counter

Counter(new_y_pred.sum(1))

### f1 micro avg is 0.47. 

In [ ]:
print(classification_report(y_test, 
                            new_y_pred, 
                            output_dict=False,
                            target_names=classes))

In [ ]:
# np.sum(y_test * new_y_pred)/len(y_test)

In [ ]:
# new_y_pred.sum(0).astype(int), np.unique(new_y_pred.sum(1).astype(int))

In [ ]:
tf_model.save(f'{out_dir}/tf_model')

### In order to choose similar moview, we compute predictions for each movie with revenue > $10 million.

In [ ]:
text_input_pred = tf_model.predict(text_input)

In [ ]:
mask = df['revenue'] > 1e7
len(mask)

In [ ]:
df2 = df[mask]
text_input_pred2 = text_input_pred[mask]

In [ ]:
out_data = df2[['name', 'year', 'genres']].to_dict(orient='list')
out_data['pred'] = text_input_pred2

out_data.keys()

In [ ]:
pkl.dump([classes, out_data], open(f'{out_dir}/aux_data.pkl', 'wb'))

### checking that spoiler database also produces a good f1 micro average

In [ ]:
df_imdb = pkl.load(open(f'{tmp_dir}/df_imdb.pkl', 'rb'))
df_imdb

In [ ]:
imdb_labels = mlb.transform(df_imdb['genre'])

In [ ]:
y_pred = tf_model.predict(df_imdb['clean'].tolist())

In [ ]:
new_y_pred = np.where((y_pred.T/y_pred.max(1)).T > 0.95, 1, 0)

In [ ]:
# from collections import Counter

Counter(new_y_pred.sum(1))

In [ ]:
print(classification_report(imdb_labels, 
                            new_y_pred, 
                            output_dict=False,
                            target_names=classes))

In [ ]:
# single_genre = lambda y_pred : np.floor((y_pred.T/y_pred.T.max(0)).T).astype(int)

In [ ]:
# %%time 

# bins = np.arange(df_feat.min().min(), df_feat.max().max(), 0.1)

# df_dig = df_feat.apply(lambda x : np.digitize(x, bins))
# df_dig[df_feat.isna()] = 0

# # len(bins)

# input_layer = ak.Input()

# x = input_layer

# x = ak.Embedding(max_features=len(bins)+1, embedding_dim=None, pretraining='random')(x)

# x = ak.RNNBlock()(x)

# # x = ak.DenseBlock()(x)

# x = ak.ClassificationHead()(x)

# rnn_mod = ak.AutoModel(
#     inputs=[input_layer],
#     outputs=[x],
#     project_name='rnn_model',
#     max_trials=1,
#     seed=SEED,
#     overwrite=True
# )

# hist = rnn_mod.fit(df_dig.to_numpy(), y_true, 
#                    validation_split=0.2,
#                    epochs=100, 
#                    batch_size=128)

# rnn_mod.predict(df_dig.to_numpy()[:10])

In [ ]:
# df['Plx'].apply(lambda x : np.log10()).hist(bins=100, log=False)

In [ ]:
# df[df['Plx']>15]['True_Class']